In [1]:
import pandas as pd
import os, re, csv

# there are total of 65 xlsx files in data folder
# collect 'T/F' and 'Tweet Text' data from both xxx.tweets and xxx.tweets.TRUE sheets from each file
# write the final collected data into onebig.csv file

dfs = []
for fname in os.listdir("/Users/nathanlin/2019spamClassify/data"):
    if re.search(r'\.xlsx$', fname):
        #print(fname)
        file_name = 'data/' + fname
        xls = pd.ExcelFile(file_name)
        for sheetname in [xls.sheet_names[0], xls.sheet_names[1]]:
            df = pd.read_excel(file_name, sheet_name=sheetname)
            dfs.append(df)

# this step is to generate onebig csv file from 2 columns: 'T/F' and 'Tweet Text'
csv_arr = []
for df in dfs:
    # get "T/F" location
    answer_index = (list(df)).index("T/F")
    # get "Tweet Text" location
    text_index = (list(df)).index("Tweet Text")
    for index, row in df.iterrows():
        # skip 1st row as it is header
        if index == 0:
            continue

        answer = row[answer_index]
        text = row[text_index]
        # if 'T/F' value is 0, convert to 'spam'
        # if 'T/F' value is 1, convert to 'ham'
        if answer == 0:
            answer = 'spam'
        else:
            answer = 'ham'
        csv_arr.append([answer, text])
print(len(csv_arr))

# write csv
with open('onebig.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['v1', 'v2'])

    for row in csv_arr:
        spamwriter.writerow(row)
        

22492


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import *
from sklearn.linear_model import *
from sklearn.svm import *
import pandas
import csv

data = pandas.read_csv('onebig.csv', sep=',', header=0, encoding='latin-1')
#print(data.columns.tolist())

# use ~4/5 of data for training(18000)
train_data = data[:18000] 
test_data = data[18000:] 
print('train data:')
print((train_data.v1).value_counts())
print('test data:')
print(((test_data.v1).value_counts()))
#classifier = OneVsRestClassifier(SVC(kernel='linear'))
classifier = OneVsRestClassifier(LogisticRegression())
vectorizer = TfidfVectorizer()

# train
vectorize_text = vectorizer.fit_transform(train_data.v2.values.astype('U'))
classifier.fit(vectorize_text, train_data.v1)

# score
vectorize_text = vectorizer.transform(test_data.v2.values.astype('U'))
score = classifier.score(vectorize_text, test_data.v1)
print('score: {}'.format(score))


csv_arr = []
print(len(test_data))
for index, row in test_data.iterrows():
    answer = row[0]
    text = row[1]

    if not isinstance(text, str):
        print(text)
        print(type(text))
        print(index)
        print(row)
        continue

    vectorize_text = vectorizer.transform([text])
    predict = classifier.predict(vectorize_text)[0]
    if predict == answer:
        result = 'right'
    else:
        result = 'wrong'
    csv_arr.append([len(csv_arr), text, answer, predict, result])


# write csv
with open('test_score.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=';',
            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['#', 'text', 'answer', 'predict', result])

    for row in csv_arr:
        spamwriter.writerow(row)

train data:
ham     10243
spam     7757
Name: v1, dtype: int64
test data:
spam    2627
ham     1865
Name: v1, dtype: int64


/anaconda3/envs/myenv3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


score: 0.832146037399822
4492


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics, svm
from sklearn.model_selection import (
    train_test_split, learning_curve, StratifiedShuffleSplit, GridSearchCV,
    cross_val_score)
import pandas
import csv
import re

def preprocess_text(messy_string):
    if not isinstance(messy_string, str):
        return ' '
    cleaned = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', messy_string)
    cleaned = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr',
                     cleaned)
    cleaned = re.sub(r'£|\$', 'moneysymb', cleaned)
    cleaned = re.sub(
        r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b',
        'phonenumbr', cleaned)
    cleaned = re.sub(r'\d+(\.\d+)?', 'numbr', cleaned)
    cleaned = re.sub(r'[^\w\d\s]', ' ', cleaned) # Remove punctuation
    cleaned = re.sub(r'\s+', ' ', cleaned) # Replace whitespace between terms with a single space
    cleaned = re.sub(r'^\s+|\s+?$', '', cleaned.lower()) # Remove leading and trailing whitespace
    return cleaned

data = pandas.read_csv('onebig.csv', sep=',', header=0)
#print(data.columns.tolist())

# Encode the class labels as numbers
le = LabelEncoder()
y_enc = le.fit_transform(data.v1)

# Store the Tweet text data
raw_text = data.v2
print(raw_text.head())
processed = raw_text.apply(preprocess_text)
print(processed.head())


# Construct a design matrix using an n-gram model and a tf-idf statistics
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_ngrams = vectorizer.fit_transform(processed)

print(X_ngrams.shape)


# Prepare the training and test sets using an 80/20 split
X_train, X_test, y_train, y_test = train_test_split(
    X_ngrams,
    y_enc,
    test_size=0.2,
    random_state=42,
    stratify=y_enc
)

# Train SVM with a linear kernel on the training set
clf = svm.LinearSVC(loss='hinge')
clf.fit(X_train, y_train)

# Evaluate the classifier on the test set
y_pred = clf.predict(X_test)

# Compute the F1 score
print(metrics.f1_score(y_test, y_pred))


# Display a confusion matrix
pandas.DataFrame(
    metrics.confusion_matrix(y_test, y_pred),
    index=[['actual', 'actual'], ['spam', 'ham']],
    columns=[['predicted', 'predicted'], ['spam', 'ham']]
)

0    I added a video to a @YouTube playlist https:/...
1    I liked a @YouTube video https://t.co/kFBopyvZ...
2    Antifa Plan Civil War To Overthrow the Governm...
3    Antifa Plan Civil War To Overthrow the Governm...
4    Exclusive: Judi Dench, who had a Harvey Weinst...
Name: v2, dtype: object
0    i added a video to a youtube playlist httpaddr...
1    i liked a youtube video httpaddr antifa plan c...
2    antifa plan civil war to overthrow the governm...
3    antifa plan civil war to overthrow the governm...
4    exclusive judi dench who had a harvey weinstei...
Name: v2, dtype: object
(22492, 63610)
0.9511737089201877


/anaconda3/envs/myenv3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


predicted      
                 spam   ham
actual spam      2265   157
       ham         51  2026